<a href="https://colab.research.google.com/github/ranabilal09/Reaserch-Assistent/blob/main/Reaserch_Assistent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!git clone  https://github.com/ranabilal09/Reaserch-Assistent.git

fatal: destination path 'Reaserch-Assistent' already exists and is not an empty directory.


In [2]:
!pip install langchain langgraph langchain-ollama langchain_community langchain-google-genai langserve arxiv duckduckgo-search

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.7 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=ea4a52c39aa33bc53836f081be8103ee760bf6468c12c6c50c9adab8f038e997
  Stored in directory: 

In [3]:
from google.colab import userdata , drive
import os
drive.mount('/content/drive')

api = userdata.get('Gemini_Api_Key')
os.environ['GOOGLE_API_KEY'] = api
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ["LANGCHAIN_API_KEY"] = userdata.get('langchai_api_key')
os.environ["LANGCHAIN_PROJECT"] = 'Research-Assistent'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from bs4 import BeautifulSoup
import requests
import json
from langchain.utilities import DuckDuckGoSearchAPIWrapper

ddg_search = DuckDuckGoSearchAPIWrapper()

def web_serarch(query:str , num_results:int = 3):
  results = ddg_search.results(query , num_results)
  return [r["link"] for r in results]



#prompt
Template = """ {text}

-------------------------

Using the above context , answer in short the following question:

> {question}

If the question cannot be anwsered using the text , imply summarize the text. Include all the factual information , numbers and states etc, if available. """
summary_prompt = ChatPromptTemplate.from_template(Template)

url = "https://blog.langchain.dev/announcing-langsmith/"

def scrape_url(url):
  try:
    response = requests.get(url)
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      page_text = soup.get_text(separator=" " , strip = True)
      return page_text
    else:
      return f"failed to retrive the websearch , request status code:{response.status_code}"
  except Exception as e:
    print(e)
    return f"Failed to retrieve the web search , An error occurred: {str(e)}"

scrape_summarize_Chain = RunnablePassthrough.assign(
    summary=RunnablePassthrough.assign(
    text = lambda x: scrape_url(x["url"])[:10000]
) | summary_prompt | ChatGoogleGenerativeAI(model= "gemini-1.5-flash-8b") | StrOutputParser()
) | (lambda x: f"URL:{x['url']}\n\n summary:{x['summary']}")

web_serarch_chain = RunnablePassthrough.assign(
    urls = lambda x: web_serarch(x["question"])
) | (lambda x: [{"question":x["question"],"url":url}for url in x["urls"]]) | scrape_summarize_Chain.map()

# #arxiv
# from langchain_community.retrievers import ArxivRetriever

# retriever = ArxivRetriever()

# #prompt
# Template = """ {docs}

# -------------------------

# Using the above context , answer in short the following question:

# > {question}

# If the question cannot be anwsered using the text , imply summarize the text. Include all the factual information , numbers and states etc, if available. """
# Summary_prompt = ChatPromptTemplate.from_template(Template)

# scrape_summarize_Chain = RunnablePassthrough.assign(
#     summary= Summary_prompt | ChatGoogleGenerativeAI(model= "gemini-1.5-flash-8b") | StrOutputParser()
# ) | (lambda x: f"Title:{x['docs'].metadata['Title']}\n\n summary:{x['summary']}")

# web_serarch_chain = RunnablePassthrough.assign(
#     docs = lambda x: retriever.get_summaries_as_docs(x["question"])
# ) | (lambda x: [{"question":x["question"],"docs":doc}for doc in x["docs"]]) | scrape_summarize_Chain.map()

system_prompt = ChatPromptTemplate.from_messages(
    [
        ("user",
         "Write 3 google search queries to serch online that form an"
         "objective opinion from the following:{question}"
         "you must response in the list of string in the following pattern:"
         '["query1","query2","query3"]')
    ]
)

system_question_chain = system_prompt | ChatGoogleGenerativeAI(model= "gemini-1.5-flash-8b") | StrOutputParser() | json.loads

full_research_chain = system_question_chain | (lambda x:[{"question":q}for q in x]) | web_serarch_chain.map()


In [5]:

#final prompt
system_template = "You are AI critical thinker research assistent.Your sole purpose is to write well written , critically acclaimed, objective and structured reports on given text."
user_template = """Information:
--------
{research_summary}
--------
Using the above information, answer the following question or topic: "{question}" in a detailed report -- \
The report should focus on the answer to the question, should be well structured, informative, \
in depth, with facts and numbers if available and a minimum of 1,200 words.
You should strive to write the report as long as you can using all relevant and necessary information provided.
You must write the report with markdown syntax.
You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
Write all used source urls at the end of the report, and make sure to not add duplicated sources, but only one reference for each.
You must write the report in apa format.
Please do your best, this is very important to my career."""

writter_prompt = ChatPromptTemplate.from_messages([
    ("system",system_template),
    ("user", user_template)
])

def collapse_list_of_lists(list_of_lists):
    flat_list = []
    for sublist in list_of_lists:
        flat_list.extend("\n\n".join(sublist))
    return flat_list

chain = RunnablePassthrough.assign(
    research_summary= full_research_chain | collapse_list_of_lists
) | writter_prompt | ChatGoogleGenerativeAI(model= "gemini-1.5-flash-8b") | StrOutputParser()



In [6]:
chain.invoke({"question":"Who is typically older: point guards and centers?"})

"# NBA Player Age Comparison: Point Guards vs. Centers\n\n**Executive Summary:**\n\nBased on the provided text snippets, centers are generally older than point guards in the NBA.  While the data does not present a comprehensive statistical comparison of average ages across all seasons, multiple sources highlight the trend of centers having a higher average age than point guards. This difference in age distribution likely reflects the distinct physical demands and skill sets required for each position.\n\n**Introduction:**\n\nThis report analyzes the provided text snippets to determine whether point guards or centers tend to be older in the NBA.  The information focuses on various aspects of NBA player demographics, but a direct comparison of average ages between point guards and centers is not always explicitly stated.  The report will synthesize the available information to draw a conclusion about the typical age difference between these two key positions.\n\n**Data Analysis and Findi

In [ ]:

!pip install --upgrade langserve fastapi uvicorn sse_starlette python-dotenv

In [ ]:
from fastapi import FastAPI
import uvicorn
from langserve import add_routes

import nest_asyncio

# This is the crucial fix for Jupyter/Colab environments
nest_asyncio.apply()

app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple api server using Langchain's Runnable interfaces",
)

add_routes(app, chain, path="/research")

if  __name__ == "__main__":
  uvicorn.run(app,host="localhost", port=5000)